# Lake Model

The `Lake` class models a lake or reservoir with a constant surface area. It tracks the water volume and level, accounting for inflows, outflows, and evaporation. This notebook provides an overview of the `Lake` model and a simulation of its behavior.

## State Variables

The state of the lake is defined by:

- `volume` (float): The current volume of water in the lake (m^3).
- `water_level` (float): The water level, calculated from the volume and surface area (m).
- `outflow` (float): The outflow from the lake (m^3/s), which is typically determined by a downstream component.

## Parameters

The physical properties of the lake are set using these parameters:

- `surface_area` (float): The surface area of the lake (m^2).
- `max_volume` (float): The maximum storage capacity of the lake (m^3).
- `evaporation_rate_m_per_s` (float): The rate of water evaporation from the lake's surface (m/s).

## Simulation Example

The code below simulates a lake over time. It starts with an initial volume and applies a constant inflow. The outflow is varied during the simulation to demonstrate its effect on the lake's volume and water level. The results are then plotted.

**Note:** To run this notebook, ensure you have `matplotlib` installed (`pip install matplotlib`) and that you are running the Jupyter server from the root directory of the project.

In [ ]:
import matplotlib.pyplot as plt
from swp.simulation_identification.physical_objects.lake import Lake
from swp.core.interfaces import State, Parameters

# Initial state and parameters for the lake
initial_state = State(volume=5e6, water_level=0.0, outflow=0.0)
parameters = Parameters(
    surface_area=1e6,      # 1 km^2
    max_volume=10e6,       # 10 million m^3
    evaporation_rate_m_per_s=1e-8 # approx. 0.864 mm/day
)

# Create a Lake instance
lake = Lake(name="main_lake", initial_state=initial_state, parameters=parameters)

# Simulation settings
dt = 3600  # time step in seconds (1 hour)
simulation_duration = 3600 * 24 * 5  # 5 days
num_steps = int(simulation_duration / dt)
inflow_rate = 10.0  # constant inflow in m^3/s

# Store results for plotting
history = []

# Run the simulation loop
for t in range(num_steps):
    # Assume outflow is requested by a downstream user
    if t > num_steps // 2:
        requested_outflow = 15.0 # Increased demand
    else:
        requested_outflow = 5.0 # Normal demand
        
    action = {'outflow': requested_outflow}
    
    lake.set_inflow(inflow_rate)
    current_state = lake.step(action=action, dt=dt)
    history.append(current_state.copy())

# Print the final state
print("Final State:", lake.get_state())

# Prepare data for plotting
time_in_hours = [i * dt / 3600 for i in range(num_steps)]
volumes = [s['volume'] for s in history]
water_levels = [s['water_level'] for s in history]
outflows = [s['outflow'] for s in history]

# Plot the results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

ax1.plot(time_in_hours, [v / 1e6 for v in volumes], label='Volume')
ax1.set_ylabel('Volume (million m^3)')
ax1.set_title('Lake Simulation')
ax1.grid(True)
ax1.legend()

ax2.plot(time_in_hours, water_levels, label='Water Level', color='orange')
ax2.set_xlabel('Time (hours)')
ax2.set_ylabel('Water Level (m)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()
